In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, runniЃng this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("D:/PythonCode/datasets/plantdisease-tugasakhir3/train"))
print(os.listdir("D:/PythonCode/datasets/resnet50"))

# Any results you write to the current directory are saved as output.

['Apple___Apple_scab', 'Apple___Black_rot', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Early_blight', 'Potato___Late_blight', 'Tomato___Bacterial_spot', 'Tomato___Septoria_leaf_spot', 'Tomato___Target_Spot']
['imagenet_class_index.json', 'resnet50_weights_tf_dim_ordering_tf_kernels.h5', 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5']


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

resnet_weights_path = 'D:/PythonCode/datasets/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [3]:
data_generator = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )

image_size = 224
batch_size = 10
train_generator = data_generator.flow_from_directory(
        'D:/PythonCode/datasets/plantdisease-tugasakhir3/train/',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

num_classes = len(train_generator.class_indices)

Found 5000 images belonging to 10 classes.


In [4]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
count = sum([len(files) for r, d, files in os.walk("D:/PythonCode/datasets/plantdisease-tugasakhir3/train/")])

model.fit_generator(
        train_generator,
        steps_per_epoch=int(count/batch_size) + 1,
        epochs=5)

Epoch 1/5
501/501 [==============================] - 7009s 14s/step - loss: 0.9260 - accuracy: 0.7802
Epoch 2/5
501/501 [==============================] - 7033s 14s/step - loss: 0.5324 - accuracy: 0.8393
Epoch 3/5
501/501 [==============================] - 6986s 14s/step - loss: 0.3654 - accuracy: 0.8800
Epoch 4/5
501/501 [==============================] - 7026s 14s/step - loss: 0.3538 - accuracy: 0.8834
Epoch 5/5
501/501 [==============================] - 7008s 14s/step - loss: 0.3269 - accuracy: 0.8910
